# Library Import

In [1]:
import requests
import pandas as pd
from datetime import date, timedelta
import numpy as np
import re
import logging
import oracledb

# Defining variable and parameters

In [2]:
#defining period metadata. This is assumed
period_metadata_df = pd.DataFrame(
    {"period" : [1, 2, 3, 4, 5],
    "periodName" : ["1st Period", "2nd Period", "3rd Period", "Overtime", "Shootout"]}
)

def underscore_to_camelcase(text):
    return re.sub(r'_([a-zA-Z])', lambda match: match.group(1).upper(), text)

def camelcase_to_underscores(text):
    # Insert an underscore before each uppercase letter, except the first one
    result = re.sub(r'(?<!^)([A-Z])', r'_\1', text)
    # Convert the result to lowercase
    return result.lower()

# Team Metadata

In [3]:
def nhl_team_metadata():
    #team metadata url
    url = 'https://api.nhle.com/stats/rest/en/team'

    # Fetch data from the URL
    response = requests.get(url)
    data = response.json()

    # Extract the 'data' field from the response
    data = data.get("data", [])

    # Create a DataFrame
    df = pd.DataFrame(data)
    df.rename({'id': 'teamId'}, axis = 1, inplace = True)
    
    return df

team_df = nhl_team_metadata()
# team_df

In [4]:
def read_config_file(file_path):
    config = {}
    with open(file_path, 'r') as file:
        for line in file:
            key, *value_parts = line.strip().split('=')
            value = '='.join(value_parts)
            config[key] = value
    return config

# Replace 'path/to/config.txt' with the actual file path
config_file_path = '/Users/jeremybutt/Desktop/credentials.txt'

# Read the configuration from the file
config = read_config_file(config_file_path)

# Set the variables for username, password, and DSN
username = config.get('Oracle_Username')
password = config.get('Oracle_Password')
dsn = config.get('dsn')

def insert_into_load_data(df, table_name, username, password, dsn):
    try:
        # Convert all dataframe columns to a string
        df = df.astype(str)
        
        # Renaming columns with camelCase
        df.rename(columns=camelcase_to_underscores, inplace=True)
        
        # Construct the connection string
        connection = oracledb.connect(user=username, password=password, dsn=dsn)

        # Create a cursor object
        cursor = connection.cursor()

#         # SQL statement to create the table
#         create_table_statement = f"""
#             CREATE TABLE {table_name} (
#                 {", ".join([f"{col} VARCHAR2(4000)" for col in df.columns])}
#             )
#         """

#         # Create the table
#         cursor.execute(create_table_statement)
#         print(f"Created table {table_name}")

        # Convert dataframe to a list of tuples
        data_tuples = [tuple(row) for row in df.values]

        # Prepare the INSERT query with placeholders for the columns
        insert_statement = f"INSERT INTO {table_name} " \
                           f"({', '.join(df.columns)}) " \
                           f"VALUES ({', '.join([':' + str(i+1) for i in range(len(df.columns))])})"

        # Insert data into the target table
        cursor.executemany(insert_statement, data_tuples)
        print(f"Inserted {len(data_tuples)} rows into {table_name}")
            
        # Call the stored procedure
        cursor.callproc('update_team_dimension')

        # Commit the transaction
        connection.commit()

        print("Stored procedure executed successfully!")

    finally:
        # Close the cursor and connection
        cursor.close()
        connection.close()
        print("Oracle connection is closed")
        
test = insert_into_load_data(team_df, 'DIM_TEAM_LOAD', username, password, dsn)
test

Inserted 59 rows into DIM_TEAM_LOAD
Stored procedure executed successfully!
Oracle connection is closed


# Game data

In [4]:
#Choose the date range for the games to return
today = date.today().strftime('%Y-%m-%d')
today_minus_7 = date.today() - timedelta(days = 7)
today_minus_7 = today_minus_7.strftime('%Y-%m-%d')

#Choose the seasonId for the game to return
season_id = 20232024

def game_data_function(seasonId):
    try:
        #url for the game data
        url = "https://api.nhle.com/stats/rest/en/game"

        # Fetch data from the URL
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
        data = response.json()

        # Extract the 'data' field from the response
        data = data.get("data", [])

        # Create a DataFrame
        df = pd.DataFrame(data)

        # Convert 'gameDate' column to datetime format
        df['gameDate'] = pd.to_datetime(df['gameDate'])
        
        # Convert 'easternStartTime' column to datetime format
        df['easternStartTime'] = pd.to_datetime(df['easternStartTime'], format="%Y-%m-%dT%H:%M:%S")

        # Filtered for seasonid for current season
        df = df[df['season'] == seasonId]

        # rename id column to gameId
        df.rename({'id': 'gameId', 'season': 'seasonId'}, axis=1, inplace=True)

        # join data to team_df to provide readable names to home team
        df = df.merge(team_df, how='left', left_on='homeTeamId', right_on='teamId')

        # drop columns from the team_df because they are not necessary
        cols_to_drop = ['teamId', 'franchiseId', 'leagueId', 'triCode']
        df.drop(columns=cols_to_drop, axis=1, inplace=True)

        # renaming columns to describe home team
        cols_to_rename = {'fullName': 'homeTeamFullName', 'rawTricode': 'homeTeamAbrv'}
        df.rename(cols_to_rename, axis=1, inplace=True)

        # join data to team_df to provide readable names to visiting team
        df = df.merge(team_df, how='left', left_on='visitingTeamId', right_on='teamId')

        # drop columns from the team_df because they are not necessary
        df.drop(columns=cols_to_drop, axis=1, inplace=True)

        # renaming columns to describe visiting team
        cols_to_rename = {'fullName': 'visitingTeamFullName', 'rawTricode': 'visitingTeamAbrv'}
        df.rename(cols_to_rename, axis=1, inplace=True)

        # creating a column to define if the home team or visiting team won the game
        df['winner'] = np.where(df['homeScore'] > df['visitingScore'], 'Home Team', 
                        np.where(df['homeScore'] < df['visitingScore'], 'Visiting Team', 'Tie'))

        # creating a column of the teamId of the winning team
        df['winnerTeamId'] = np.where(df['homeScore'] > df['visitingScore'], df['homeTeamId'], 
                                np.where(df['homeScore'] < df['visitingScore'], df['visitingTeamId'], None))

        # creating a column with the fullName of the winning team
        df = df.merge(team_df[['teamId', 'fullName', 'rawTricode']], how='left', left_on='winnerTeamId', right_on='teamId')

        # dropping teamId as it is redundant
        cols_to_drop = ['teamId']
        df.drop(columns=cols_to_drop, axis=1, inplace=True)

        # renaming columns from team_df to describe the column
        cols_to_rename = {'fullName': 'winnerTeamFullName', 'rawTricode': 'winnerTeamAbrv'}
        df.rename(cols_to_rename, axis=1, inplace=True)

        # adding readable column about the current period or the period the game ended in
        df = df.merge(period_metadata_df, how="left", on="period")
        
        df = df[['gameId',
                 'seasonId',
                 'easternStartTime',
                 'gameDate',
                 'gameNumber',
                 'gameScheduleStateId',
                 'gameStateId',
                 'gameType',
                 'period',
                 'periodName',
                 'homeScore',
                 'homeTeamId',
                 'homeTeamFullName',
                 'homeTeamAbrv',
                 'visitingScore',
                 'visitingTeamId',
                 'visitingTeamFullName',
                 'visitingTeamAbrv',
                 'winner',
                 'winnerTeamId',
                 'winnerTeamFullName',
                 'winnerTeamAbrv']]
        
        return df
    
    except Exception as e:
        print(f"An error occurred: {e}")
        return None  # Return None to indicate failure


game_data = game_data_function(season_id)
# print(game_data.head())
# Display the filtered DataFrame
# print(game_data[game_data['gameId']==2023020849].head())

# Code to insert data into Oracle database

In [6]:
def read_config_file(file_path):
    config = {}
    with open(file_path, 'r') as file:
        for line in file:
            key, *value_parts = line.strip().split('=')
            value = '='.join(value_parts)
            config[key] = value
    return config

# Replace 'path/to/config.txt' with the actual file path
config_file_path = '/Users/jeremybutt/Desktop/credentials.txt'

# Read the configuration from the file
config = read_config_file(config_file_path)

# Set the variables for username, password, and DSN
username = config.get('Oracle_Username')
password = config.get('Oracle_Password')
dsn = config.get('dsn')

def insert_into_load_data(df, table_name, username, password, dsn):
    try:
        # Convert all dataframe columns to a string
        df = df.astype(str)
        
        # Renaming columns with camelCase
        df.rename(columns=camelcase_to_underscores, inplace=True)
        
        # Construct the connection string
        connection = oracledb.connect(user=username, password=password, dsn=dsn)

        # Create a cursor object
        cursor = connection.cursor()

        # SQL statement to create the table
#         create_table_statement = f"""
#             CREATE TABLE {table_name} (
#                 {", ".join([f"{col} VARCHAR2(4000)" for col in df.columns])}
#             )
#         """

#         # Create the table
#         cursor.execute(create_table_statement)
#         print(f"Created table {table_name}")

        # Convert dataframe to a list of tuples
        data_tuples = [tuple(row) for row in df.values]

        # Prepare the INSERT query with placeholders for the columns
        insert_statement = f"INSERT INTO {table_name} " \
                           f"({', '.join(df.columns)}) " \
                           f"VALUES ({', '.join([':' + str(i+1) for i in range(len(df.columns))])})"

        # Insert data into the target table
        cursor.executemany(insert_statement, data_tuples)
        print(f"Inserted {len(data_tuples)} rows into {table_name}")
            
        # Call the stored procedure
        cursor.callproc('update_fact_game')

        # Commit the transaction
        connection.commit()

        print("Stored procedure executed successfully!")
    
    except oracledb.DatabaseError as e:
        # Rollback any uncommitted changes if an error occurs
        connection.rollback()
        # Print out the exception message
        print("An error occurred while executing the stored procedure:", e)

    finally:
        # Close the cursor and connection
        cursor.close()
        connection.close()
        print("Oracle connection is closed")
        
insert_into_load_data(game_data, 'FACT_GAME_LOAD', username, password, dsn)

Inserted 1427 rows into FACT_GAME_LOAD
Stored procedure executed successfully!
Oracle connection is closed


# Loop to download game data

In [ ]:
#Choose the seasonId for the game to return
years = range(1919, 2020, 10)
for year in years:
    def read_config_file(file_path):
        config = {}
        with open(file_path, 'r') as file:
            for line in file:
                key, *value_parts = line.strip().split('=')
                value = '='.join(value_parts)
                config[key] = value
        return config

    # Replace 'path/to/config.txt' with the actual file path
    config_file_path = '/Users/jeremybutt/Desktop/credentials.txt'

    # Read the configuration from the file
    config = read_config_file(config_file_path)

    # Set the variables for username, password, and DSN
    username = config.get('Oracle_Username')
    password = config.get('Oracle_Password')
    dsn = config.get('dsn')
    
#     print("season_id: " + str(year) + str(year+1))
    season_id = pd.Series(int(str(year) + str(year+1)))
    print(season_id)

    game_data = game_data_function(season_id[0])
    
#     print(game_data.head())

    insert_into_load_data(game_data, 'FACT_GAME_LOAD', username, password, dsn)

# Example of filtering game data

In [ ]:
# game_data[(game_data['visitingTeamId'] == 5) & (game_data['homeTeamId'] == 29) & (game_data['gameDate'] == '2023-09-24')]
# game_data[game_data['gameType'] != 1]
print(game_data[game_data['gameDate'] == '20240206'])

# Boxscore Data

In [ ]:
#defining gameId
game_id = '2023020837'

def game_boxscore_data(gameId):
    # defining URL for boxscore data
    url = 'https://api-web.nhle.com/v1/gamecenter/' + str(gameId) + '/boxscore'

    # Fetch data from the URL
    response = requests.get(url)
    data = response.json()

    # Extract the 'data' field from the response
    data = pd.json_normalize(data)

    #####away team boxscore#####
    #extracting the away team info from the game boxscore
    away_team_meta = data.loc[:, ['awayTeam.id', 'awayTeam.name.default', 'awayTeam.abbrev']]

    #columns to rename
    cols_to_rename = {'awayTeam.id': 'teamId', 'awayTeam.name.default': 'teamName', 'awayTeam.abbrev': 'teamAbbrev'}
    away_team_meta.rename(cols_to_rename, axis = 1, inplace = True)

    #extracting boxscore data by forwards and defense and unpivoting the data
    away_team_boxscore_forwards = pd.melt(pd.json_normalize(data['boxscore.playerByGameStats.awayTeam.forwards']))
    away_team_boxscore_defense = pd.melt(pd.json_normalize(data['boxscore.playerByGameStats.awayTeam.defense']))

    #combining forward and defense boxscore data into a single dataframe
    away_team_boxscore = pd.concat([away_team_boxscore_forwards, away_team_boxscore_defense])

    #data is in a dictionary. so need to normalize into a dataframe
    away_team_boxscore_df = pd.json_normalize(away_team_boxscore['value'])

    #putting team data with boxscore data to identify which team players play for
    away_team_boxscore_df = pd.concat([away_team_boxscore_df, away_team_meta], axis = 1)

    #team data only fills for first row. Need to fill all rows with data
    away_team_boxscore_df['teamId'].ffill(inplace = True)
    away_team_boxscore_df['teamName'].ffill(inplace = True)
    away_team_boxscore_df['teamAbbrev'].ffill(inplace = True)

    #creating a column with the gameId
    away_team_boxscore_df['gameId'] = game_id
    
    #droping columns if they exist
    away_team_boxscore_df = away_team_boxscore_df[away_team_boxscore_df.columns.drop(list(away_team_boxscore_df.filter(regex='^name.[a-z]{2}$')), errors = 'ignore')]

    #####home team boxscore#####
    #extracting the away team info from the game boxscore
    home_team_meta = data.loc[:, ['homeTeam.id', 'homeTeam.name.default', 'homeTeam.abbrev']]

    #columns to rename
    cols_to_rename = {'homeTeam.id': 'teamId', 'homeTeam.name.default': 'teamName', 'homeTeam.abbrev': 'teamAbbrev'}
    home_team_meta.rename(cols_to_rename, axis = 1, inplace = True)

    #extracting boxscore data by forwards and defense and unpivoting the data
    home_team_boxscore_forwards = pd.melt(pd.json_normalize(data['boxscore.playerByGameStats.homeTeam.forwards']))
    home_team_boxscore_defense = pd.melt(pd.json_normalize(data['boxscore.playerByGameStats.homeTeam.defense']))

    #combining forward and defense boxscore data into a single dataframe
    home_team_boxscore_df = pd.concat([home_team_boxscore_forwards, home_team_boxscore_defense])

    #data is in a dictionary. so need to normalize into a dataframe
    home_team_boxscore_df = pd.json_normalize(home_team_boxscore_df['value'])

    #putting team data with boxscore data to identify which team players play for
    home_team_boxscore_df = pd.concat([home_team_boxscore_df, home_team_meta], axis = 1)

    #team data only fills for first row. Need to fill all rows with data
    home_team_boxscore_df['teamId'].ffill(inplace = True)
    home_team_boxscore_df['teamName'].ffill(inplace = True)
    home_team_boxscore_df['teamAbbrev'].ffill(inplace = True)

    #creating a column with the gameId
    home_team_boxscore_df['gameId'] = game_id
    
    #droping columns if they exist
    home_team_boxscore_df = home_team_boxscore_df[home_team_boxscore_df.columns.drop(list(home_team_boxscore_df.filter(regex='^name.[a-z]{2}$')), errors = 'ignore')]
    
    #combining both away team boxscore and home team boxscore
    boxscore_df = pd.concat([away_team_boxscore_df, home_team_boxscore_df])
    boxscore_df.rename({'name.default': 'playerNameDefault'}, axis = 1, inplace = True)
    
    boxscore_df = boxscore_df[['gameId',
                               'playerId', 
                               'playerNameDefault', 
                               'teamId', 
                               'teamName',
                               'teamAbbrev',
                               'sweaterNumber', 
                               'position', 
                               'goals', 
                               'assists', 
                               'points',
                               'plusMinus', 
                               'pim', 
                               'hits', 
                               'blockedShots', 
                               'powerPlayGoals',
                               'powerPlayPoints', 
                               'shorthandedGoals', 
                               'shPoints', 
                               'shots', 
                               'faceoffs',
                               'faceoffWinningPctg']]
    
    return boxscore_df

boxscore_df = game_boxscore_data(game_id)

# Example of filtering boxscore data

In [ ]:
# boxscore_df[boxscore_df['position'] == 'C']
boxscore_df

# Boxscore Data Extract New
This is a cleaner way to extract boxscore data. Less redundant code

In [5]:
def extract_team_boxscore(data, team_type):
    team_meta = data.loc[:, [f'{team_type}Team.id', f'{team_type}Team.name.default', f'{team_type}Team.abbrev']]
    cols_to_rename = {f'{team_type}Team.id': 'teamId', f'{team_type}Team.name.default': 'teamName', f'{team_type}Team.abbrev': 'teamAbbrev'}
    team_meta.rename(cols_to_rename, axis=1, inplace=True)
    
    # Convert DataFrame to dictionary without index
    team_meta = team_meta.to_dict(orient='records')[0]

    team_boxscore_forwards = pd.melt(pd.json_normalize(data[f'boxscore.playerByGameStats.{team_type}Team.forwards']))
    team_boxscore_defense = pd.melt(pd.json_normalize(data[f'boxscore.playerByGameStats.{team_type}Team.defense']))
    
    team_boxscore_df = pd.concat([team_boxscore_forwards, team_boxscore_defense])
    team_boxscore_df = pd.json_normalize(team_boxscore_df['value']).astype(str)
    
    # Adding new columns from the dictionary
    for key, value in team_meta.items():
        team_boxscore_df[key] = value

    team_boxscore_df = team_boxscore_df[team_boxscore_df.columns.drop(list(team_boxscore_df.filter(regex='^name.[a-z]{2}$')), errors='ignore')]
    
    return team_boxscore_df

def extract_goalie_boxscore(data, team_type):
    team_meta = data.loc[:, [f'{team_type}Team.id', f'{team_type}Team.name.default', f'{team_type}Team.abbrev']]
    cols_to_rename = {f'{team_type}Team.id': 'teamId', f'{team_type}Team.name.default': 'teamName', f'{team_type}Team.abbrev': 'teamAbbrev'}
    team_meta.rename(cols_to_rename, axis=1, inplace=True)
    # Convert DataFrame to dictionary without index
    team_meta = team_meta.to_dict(orient='records')[0]

    team_boxscore_goalie = pd.melt(pd.json_normalize(data[f'boxscore.playerByGameStats.{team_type}Team.goalies']))
    goalie_boxscore_df = pd.json_normalize(team_boxscore_goalie['value'])

    # Adding new columns from the dictionary
    for key, value in team_meta.items():
        goalie_boxscore_df[key] = value

    goalie_boxscore_df = goalie_boxscore_df[goalie_boxscore_df.columns.drop(list(goalie_boxscore_df.filter(regex='^name.[a-z]{2}$')), errors='ignore')]

    return goalie_boxscore_df

def player_boxscore_data(game_id):
    url = 'https://api-web.nhle.com/v1/gamecenter/' + str(game_id) + '/boxscore'
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        json_data = response.json()
    except requests.RequestException as e:
        logging.error(f"Error fetching data: {e}")
        return None
    
    # Extract the 'data' field from the response
    data = pd.json_normalize(json_data)

    away_team_boxscore_df = extract_team_boxscore(data, 'away')
    home_team_boxscore_df = extract_team_boxscore(data, 'home')

    boxscore_df = pd.concat([away_team_boxscore_df, home_team_boxscore_df], ignore_index = True)

    boxscore_df.rename({'name.default': 'playerNameDefault'}, axis = 1, inplace = True)
    
    boxscore_df['gameId'] = game_id

    boxscore_df = boxscore_df[['gameId',
                               'playerId', 
                               'playerNameDefault', 
                               'teamId', 
                               'teamName',
                               'teamAbbrev',
                               'sweaterNumber', 
                               'position', 
                               'goals', 
                               'assists', 
                               'points',
                               'plusMinus', 
                               'pim', 
                               'hits', 
                               'blockedShots', 
                               'powerPlayGoals',
                               'powerPlayPoints', 
                               'shorthandedGoals', 
                               'shPoints', 
                               'shots', 
                               'faceoffs',
                               'faceoffWinningPctg']]

    return boxscore_df

def goalie_boxscore_data(game_id):
    url = 'https://api-web.nhle.com/v1/gamecenter/' + str(game_id) + '/boxscore'
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        json_data = response.json()
    except requests.RequestException as e:
        logging.error(f"Error fetching data: {e}")
        return None
    
    # Extract the 'data' field from the response
    data = pd.json_normalize(json_data)

    away_goalie_boxscore_df = extract_goalie_boxscore(data, 'away')
    home_goalie_boxscore_df = extract_goalie_boxscore(data, 'home')

    boxscore_df = pd.concat([away_goalie_boxscore_df, home_goalie_boxscore_df], ignore_index = True)

    boxscore_df.rename({'name.default': 'playerNameDefault'}, axis = 1, inplace = True)
    
    boxscore_df['gameId'] = game_id

    boxscore_df = boxscore_df[['gameId',
                               'playerId',
                               'playerNameDefault', 
                               'teamId', 
                               'teamName',
                               'teamAbbrev',
                               'sweaterNumber', 
                               'position', 
                               'evenStrengthShotsAgainst',
                               'powerPlayShotsAgainst', 
                               'shorthandedShotsAgainst', 
                               'saveShotsAgainst',
                               'savePctg', 
                               'evenStrengthGoalsAgainst', 
                               'powerPlayGoalsAgainst',
                               'shorthandedGoalsAgainst', 
                               'goalsAgainst', 
                               'toi']]

    return boxscore_df

In [13]:
# boxscore_df()'2023020837'
# game_data[game_data['gameId']==2022030127]
goalie_boxscore_data('2023020035')
# player_boxscore_data('2022010034')#.astype(str)
# player_boxscore_data('2023020845')
# 2023020845.0

,gameId,playerId,playerNameDefault,teamId,teamName,teamAbbrev,sweaterNumber,position,evenStrengthShotsAgainst,powerPlayShotsAgainst,shorthandedShotsAgainst,saveShotsAgainst,savePctg,evenStrengthGoalsAgainst,powerPlayGoalsAgainst,shorthandedGoalsAgainst,goalsAgainst,toi
0,2023020035,8478024,V. Husso,17,Red Wings,DET,35,G,0/0,0/0,0/0,0/0,NaN,0,0,0,0,00:00
1,2023020035,8473503,J. Reimer,17,Red Wings,DET,47,G,17/17,6/6,0/0,23/23,1.000,0,0,0,0,60:00
2,2023020035,8477484,S. Martin,29,Blue Jackets,CBJ,30,G,18/20,4/6,2/2,24/28,0.857,2,2,0,4,59:16
3,2023020035,8482982,J. Greaves,29,Blue Jackets,CBJ,73,G,0/0,0/0,0/0,0/0,NaN,0,0,0,0,00:00


# Code to insert data into Oracle Database

In [7]:
game_id = 2023020843

player_boxscore_df = player_boxscore_data(game_id)

def read_config_file(file_path):
    config = {}
    with open(file_path, 'r') as file:
        for line in file:
            key, *value_parts = line.strip().split('=')
            value = '='.join(value_parts)
            config[key] = value
    return config

# Replace 'path/to/config.txt' with the actual file path
config_file_path = '/Users/jeremybutt/Desktop/credentials.txt'

# Read the configuration from the file
config = read_config_file(config_file_path)

# Set the variables for username, password, and DSN
username = config.get('Oracle_Username')
password = config.get('Oracle_Password')
dsn = config.get('dsn')

def insert_into_load_data(df, table_name, procedure, username, password, dsn):
    try:
        # Convert all dataframe columns to a string
        df = df.astype(str)
        
        # Renaming columns with camelCase
        df.rename(columns=camelcase_to_underscores, inplace=True)
        
        # Construct the connection string
        connection = oracledb.connect(user=username, password=password, dsn=dsn)

        # Create a cursor object
        cursor = connection.cursor()

#         # SQL statement to create the table
#         create_table_statement = f"""
#             CREATE TABLE {table_name} (
#                 {", ".join([f"{col} VARCHAR2(4000)" for col in df.columns])}
#             )
#         """

#         # Create the table
#         cursor.execute(create_table_statement)
#         print(f"Created table {table_name}")

        # Convert dataframe to a list of tuples
        data_tuples = [tuple(row) for row in df.values]

        # Prepare the INSERT query with placeholders for the columns
        insert_statement = f"INSERT INTO {table_name} " \
                           f"({', '.join(df.columns)}) " \
                           f"VALUES ({', '.join([':' + str(i+1) for i in range(len(df.columns))])})"

        # Insert data into the target table
        cursor.executemany(insert_statement, data_tuples)
        #print(f"Inserted {len(data_tuples)} rows into {table_name}")
            
        # Call the stored procedure
        cursor.callproc(procedure)

        # Commit the transaction
        connection.commit()

        #print("Stored procedure executed successfully!")
    
    except oracledb.DatabaseError as e:
        # Rollback any uncommitted changes if an error occurs
        connection.rollback()
        # Print out the exception message
        print("An error occurred while executing the stored procedure:", e)

    finally:
        # Close the cursor and connection
        cursor.close()
        connection.close()
        #print("Oracle connection is closed")
        
# insert_into_load_data(df = player_boxscore_df, 
#                       table_name = 'FACT_PLAYER_BOXSCORE_LOAD', 
#                       procedure = 'update_fact_player_boxscore',
#                       username = username, 
#                       password = password, 
#                       dsn = dsn)

# Loop for Player Boxscore

In [8]:
#Choose the seasonId for the game to return
# season_id = pd.Series([20032004, 20022003, 20012002, 20002001]).reset_index(drop=True)
season_id = pd.Series(20232024)

for season in season_id:
    
    game_data = game_data_function(season)

    game_id = game_data['gameId'].reset_index(drop=True)
#     game_id = game_data[(game_data['gameId']>2001020829)&(game_data['gameDate']<'2024-02-20')]['gameId'].reset_index(drop=True)
    game_id = game_data[(game_data['gameDate']>='2024-02-20')&(game_data['gameDate']<'2024-03-02')]['gameId'].reset_index(drop=True)
    # game_id = game_data[game_data['gameDate']<'2024-02-20']['gameId'].reset_index(drop=True)
    # game_id = game_data[game_data['gameId']==2023020843]['gameId'].reset_index(drop=True)

    for game in game_id:
        try:
            def read_config_file(file_path):
                config = {}
                with open(file_path, 'r') as file:
                    for line in file:
                        key, *value_parts = line.strip().split('=')
                        value = '='.join(value_parts)
                        config[key] = value
                return config

            # Replace 'path/to/config.txt' with the actual file path
            config_file_path = '/Users/jeremybutt/Desktop/credentials.txt'

            # Read the configuration from the file
            config = read_config_file(config_file_path)

            # Set the variables for username, password, and DSN
            username = config.get('Oracle_Username')
            password = config.get('Oracle_Password')
            dsn = config.get('dsn')

            print(game)

            data = player_boxscore_data(game)
        #     print(data)

            insert_into_load_data(df = data, 
                                  table_name = 'FACT_PLAYER_BOXSCORE_LOAD', 
                                  procedure = 'update_fact_player_boxscore',
                                  username = username, 
                                  password = password, 
                                  dsn = dsn)
        except Exception as e:
            print(f"An error occurred for game {game}: {e}")            
            continue  # Skip to the next iteration

2023020876
2023020877
2023020878
2023020879
2023020880
2023020881
2023020882
2023020883
2023020884
2023020885
2023020886
2023020887
2023020888
2023020889
2023020890
2023020891
2023020892
2023020893
2023020894
2023020895
2023020896
2023020897
2023020898
2023020899
2023020900
2023020901
2023020902
2023020903
2023020904
2023020905
2023020906
2023020907
2023020908
2023020909
2023020910
2023020911
2023020912
2023020913
2023020914
2023020915
2023020916
2023020917
2023020918
2023020919
2023020920
2023020921
2023020922
2023020923
2023020924
2023020925
2023020926
2023020927
2023020928
2023020929
2023020930
2023020931
2023020932
2023020933
2023020934
2023020935
2023020936
2023020937
2023020938
2023020939
2023020940
2023020941
2023020942
2023020943
2023020944
2023020945
2023020946
2023020947
2023020948
2023020949
2023020950
2023020951
2023020952
2023020953
2023020954
2023020955


In [ ]:
# Connection credentials
def read_config_file(file_path):
    config = {}
    with open(file_path, 'r') as file:
        for line in file:
            key, *value_parts = line.strip().split('=')
            value = '='.join(value_parts)
            config[key] = value
    return config

# Replace 'path/to/config.txt' with the actual file path
config_file_path = '/Users/jeremybutt/Desktop/credentials.txt'

# Read the configuration from the file
config = read_config_file(config_file_path)

# Set the variables for username, password, and DSN
username = config.get('Oracle_Username')
password = config.get('Oracle_Password')
dsn = config.get('dsn')


# Construct the connection string
connection = oracledb.connect(user=username, password=password, dsn=dsn)

# Create a cursor object
cursor = connection.cursor()

try:
    # Your SQL query
    sql_query = """
    SELECT DISTINCT
        fg.game_id
    FROM
        fact_game fg
        LEFT JOIN fact_player_boxscore fpb
            ON fg.game_id = fpb.game_id
    WHERE
        fpb.game_id IS NULL
        AND season_id >= 20002001
    """

    # Execute the query
    cursor.execute(sql_query)

    # Fetch all rows as a list of tuples
    rows = cursor.fetchall()

    # Convert the result to a DataFrame
    columns = [col[0] for col in cursor.description]
    df = pd.DataFrame(rows, columns=columns)

    # Print the DataFrame
    print("DataFrame Result:")
    print(df)

except cx_Oracle.Error as error:
    print("Error executing query:", error)

finally:
    # Close the cursor and connection
    cursor.close()
    connection.close()


game_id = df['GAME_ID'].reset_index(drop=True)
#     game_id = game_data[(game_data['gameId']>2001020829)&(game_data['gameDate']<'2024-02-20')]['gameId'].reset_index(drop=True)
#     game_id = game_data[(game_data['gameDate']>='2024-02-20')&(game_data['gameDate']<'2024-02-25')]['gameId'].reset_index(drop=True)
# game_id = game_data[game_data['gameDate']<'2024-02-20']['gameId'].reset_index(drop=True)
# game_id = game_data[game_data['gameId']==2023020843]['gameId'].reset_index(drop=True)

for game in game_id:
    try:
        def read_config_file(file_path):
            config = {}
            with open(file_path, 'r') as file:
                for line in file:
                    key, *value_parts = line.strip().split('=')
                    value = '='.join(value_parts)
                    config[key] = value
            return config

        # Replace 'path/to/config.txt' with the actual file path
        config_file_path = '/Users/jeremybutt/Desktop/credentials.txt'

        # Read the configuration from the file
        config = read_config_file(config_file_path)

        # Set the variables for username, password, and DSN
        username = config.get('Oracle_Username')
        password = config.get('Oracle_Password')
        dsn = config.get('dsn')

        print(game)

        data = player_boxscore_data(game)
    #     print(data)

        insert_into_load_data(df = data, 
                              table_name = 'FACT_PLAYER_BOXSCORE_LOAD', 
                              procedure = 'update_fact_player_boxscore',
                              username = username, 
                              password = password, 
                              dsn = dsn)
    except Exception as e:
        print(f"An error occurred for game {game}: {e}")
        continue  # Skip to the next iteration

# Glossary Data

# Period scoring Dataframe

In [12]:
#defining gameId
game_id = '2023010001'

def period_scoring(game_id):
    # defining URL for boxscore data
    url = 'https://api-web.nhle.com/v1/gamecenter/' + str(game_id) + '/boxscore'

    # Fetch data from the URL
    response = requests.get(url)
    json_data = response.json()

    # Extract the 'data' field from the response
    df = pd.json_normalize(json_data)

    #converting the nested data into a dataframe.
    df = pd.json_normalize(df['boxscore.linescore.byPeriod'],
                              sep = "_")

    #unpivoting data
    df = pd.melt(df)

    #converting unpivoted data into a dataframe
    df = pd.json_normalize(df['value'])

    #renaming columns with camelCase
    df.rename(columns = underscore_to_camelcase, inplace = True)
    df['GameId'] = game_id
    
    return df
    
# period_scoring_df = period_scoring(game_id)
# print(period_scoring_df)

   period  away  home  periodDescriptorNumber periodDescriptorPeriodType  \
0       1     0     0                       1                        REG   
1       2     3     4                       2                        REG   
2       3     0     1                       3                        REG   

       GameId  
0  2023010001  
1  2023010001  
2  2023010001  


In [13]:
#defining gameId
game_id = '2023010001'

def period_scoring_tidy(game_id):
    # defining URL for boxscore data
    url = 'https://api-web.nhle.com/v1/gamecenter/' + str(game_id) + '/boxscore'

    # Fetch data from the URL
    response = requests.get(url)
    json_data = response.json()

    # Extract the 'data' field from the response
    df = pd.json_normalize(json_data)

    #converting the nested data into a dataframe.
    df = pd.json_normalize(df['boxscore.linescore.byPeriod'],
                                          sep = "_")

    #unpivoting data
    df = pd.melt(df)

    #converting unpivoted data into a dataframe
    df = pd.json_normalize(df['value'])

    #renaming columns with camelCase
    df.rename(columns = underscore_to_camelcase, inplace = True)
    
    df = df.melt(id_vars = ['period', 'periodDescriptorNumber', 'periodDescriptorPeriodType'],
                 value_vars = ['away', 'home'],
                 var_name = 'teamHomeAway',
                 value_name = 'goals')
    df['GameId'] = game_id
    
    return df
    
period_scoring_tidy(game_id)

,period,periodDescriptorNumber,periodDescriptorPeriodType,teamHomeAway,goals,GameId
0,1,1,REG,away,0,2023010001
1,2,2,REG,away,3,2023010001
2,3,3,REG,away,0,2023010001
3,1,1,REG,home,0,2023010001
4,2,2,REG,home,4,2023010001
5,3,3,REG,home,1,2023010001


In [14]:
#defining gameId
game_id = '2023010001'

def period_shots(game_id):
    # defining URL for boxscore data
    url = 'https://api-web.nhle.com/v1/gamecenter/' + str(game_id) + '/boxscore'

    # Fetch data from the URL
    response = requests.get(url)
    json_data = response.json()

    # Extract the 'data' field from the response
    data = pd.json_normalize(json_data)

    #converting the nested data into a dataframe.
    df = pd.json_normalize(data['boxscore.shotsByPeriod'],
                                          sep = "_")

    #unpivoting data
    df = pd.melt(df)

    #converting unpivoted data into a dataframe
    df = pd.json_normalize(df['value'])

    #renaming columns with camelCase
    df.rename(columns = underscore_to_camelcase, inplace = True)
    df['GameId'] = game_id
    
    return df

period_shots(game_id)
# del(period_shots)

,period,away,home,periodDescriptorNumber,periodDescriptorPeriodType,GameId
0,1,15,7,1,REG,2023010001
1,2,11,9,2,REG,2023010001
2,3,10,4,3,REG,2023010001


In [15]:
def period_shots_tidy(game_id):
    
    # defining URL for boxscore data
    url = 'https://api-web.nhle.com/v1/gamecenter/' + str(game_id) + '/boxscore'

    # Fetch data from the URL
    response = requests.get(url)
    json_data = response.json()

    # Extract the 'data' field from the response
    data = pd.json_normalize(json_data)

    #converting the nested data into a dataframe.
    df = pd.json_normalize(data['boxscore.shotsByPeriod'],
                                          sep = "_")

    #unpivoting data
    df = pd.melt(df)

    #converting unpivoted data into a dataframe
    df = pd.json_normalize(df['value'])

    #renaming columns with camelCase
    df.rename(columns = underscore_to_camelcase, inplace = True)
    
    df = df.melt(id_vars = ['period', 'periodDescriptorNumber', 'periodDescriptorPeriodType'],
                 value_vars = ['away', 'home'],
                 var_name = 'teamHomeAway',
                 value_name = 'shots')
    df['GameId'] = game_id
    
    return df

period_shots_tidy(game_id)

,period,periodDescriptorNumber,periodDescriptorPeriodType,teamHomeAway,shots,GameId
0,1,1,REG,away,15,2023010001
1,2,2,REG,away,11,2023010001
2,3,3,REG,away,10,2023010001
3,1,1,REG,home,7,2023010001
4,2,2,REG,home,9,2023010001
5,3,3,REG,home,4,2023010001


In [18]:
def insert_into_load_data(df, table_name, procedure, username, password, dsn):
    try:
        # Replace NaN values with None
        df = df.where(pd.notnull(df), None)
        
        # Convert all dataframe columns to a string
#         df = df.astype(str)
        
        # Renaming columns with camelCase
        df.rename(columns=camelcase_to_underscores, inplace=True)
        
        # Construct the connection string
        connection = oracledb.connect(user=username, password=password, dsn=dsn)

        # Create a cursor object
        cursor = connection.cursor()

        # SQL statement to create the table
#         create_table_statement = f"""
#             CREATE TABLE {table_name} (
#                 {", ".join([f"{col} VARCHAR2(4000)" for col in df.columns])}
#             )
#         """

#         # Create the table
#         cursor.execute(create_table_statement)
#         print(f"Created table {table_name}")

        # Convert dataframe to a list of tuples
        data_tuples = [tuple(row) for row in df.values]

        # Prepare the INSERT query with placeholders for the columns
        insert_statement = f"INSERT INTO {table_name} " \
                           f"({', '.join(df.columns)}) " \
                           f"VALUES ({', '.join([':' + str(i+1) for i in range(len(df.columns))])})"

        # Insert data into the target table
        cursor.executemany(insert_statement, data_tuples)
#         print(f"Inserted {len(data_tuples)} rows into {table_name}")
            
        # Call the stored procedure
        cursor.callproc(procedure)

        # Commit the transaction
        connection.commit()

        #print("Stored procedure executed successfully!")
    
    except oracledb.DatabaseError as e:
        # Rollback any uncommitted changes if an error occurs
        connection.rollback()
        # Print out the exception message
        print("An error occurred while executing the stored procedure:", e)

    finally:
        # Close the cursor and connection
        cursor.close()
        connection.close()
#         print("Oracle connection is closed")

In [22]:
season_id = pd.Series([20192020])

for season in season_id:
    
    game_data = game_data_function(season)

    game_id = game_data['gameId'].reset_index(drop=True)
        
    for game in game_id:
        try:
            def read_config_file(file_path):
                config = {}
                with open(file_path, 'r') as file:
                    for line in file:
                        key, *value_parts = line.strip().split('=')
                        value = '='.join(value_parts)
                        config[key] = value
                return config

            # Replace 'path/to/config.txt' with the actual file path
            config_file_path = '/Users/jeremybutt/Desktop/credentials.txt'

            # Read the configuration from the file
            config = read_config_file(config_file_path)

            # Set the variables for username, password, and DSN
            username = config.get('Oracle_Username')
            password = config.get('Oracle_Password')
            dsn = config.get('dsn')

            print(game)

            # Assuming you have a list of DataFrames, table names, and procedures of the same length
            list_of_dataframes = [period_scoring(game), period_shots(game), goalie_boxscore_data(game)]  # Replace with your list of DataFrames
            list_of_table_names = ['FACT_PERIOD_SCORE_LOAD', 'FACT_PERIOD_SHOTS_LOAD', 'FACT_GOALIE_BOXSCORE_LOAD']  # Replace with your list of table names
            list_of_procedures = ['UPDATE_FACT_PERIOD_SCORE', 'UPDATE_FACT_PERIOD_SHOTS', 'UPDATE_FACT_GOALIE_BOXSCORE']  # Replace with your list of procedures


            # Loop through each DataFrame, table name, and procedure, and execute the function
            for df, table_name, procedure in zip(list_of_dataframes, list_of_table_names, list_of_procedures):
                try:
                    insert_into_load_data(df, table_name, procedure, username, password, dsn)
                except Exception as e:
                    print(f"Error occurred for DataFrame '{table_name}' with procedure '{procedure}': {e}")
                    # You can choose to continue with the next iteration or handle the error as needed
                    continue
                
            del(list_of_dataframes)
            del(list_of_table_names)
            del(list_of_procedures)
            
        except Exception as e:
            print(f"An error occurred for game {game}: {e}")            
            continue  # Skip to the next iteration

An error occurred for game 2019010106: "['playerId', 'playerNameDefault', 'sweaterNumber', 'position', 'evenStrengthShotsAgainst', 'powerPlayShotsAgainst', 'shorthandedShotsAgainst', 'saveShotsAgainst', 'savePctg', 'evenStrengthGoalsAgainst', 'powerPlayGoalsAgainst', 'shorthandedGoalsAgainst', 'goalsAgainst', 'toi'] not in index"
An error occurred for game 2019010107: "['playerId', 'playerNameDefault', 'sweaterNumber', 'position', 'evenStrengthShotsAgainst', 'powerPlayShotsAgainst', 'shorthandedShotsAgainst', 'saveShotsAgainst', 'savePctg', 'evenStrengthGoalsAgainst', 'powerPlayGoalsAgainst', 'shorthandedGoalsAgainst', 'goalsAgainst', 'toi'] not in index"
An error occurred while executing the stored procedure: DPY-4004: invalid number
An error occurred while executing the stored procedure: DPY-4004: invalid number
An error occurred while executing the stored procedure: DPY-4004: invalid number
An error occurred while executing the stored procedure: DPY-4004: invalid number
An error occ